In [3]:
import os
import cv2
from deepface import DeepFace



# Process image folder

In [4]:
folder = './faces'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['age', 'gender', 'race', 'emotion'])
        print(file_name)
        # Acceder al primer elemento de la lista (el único elemento en este caso)
        elemento = obj[0]

        # Imprimir los valores de las claves 'region', 'age', 'gender', 'race', 'dominant_race', 'emotion', 'dominant_emotion'
        print(elemento["region"])
        print(elemento["age"])
        print(elemento["gender"])
        print(elemento["race"][elemento["dominant_race"]])
        print(elemento["dominant_race"])
        print(elemento["emotion"])
        print(elemento["dominant_emotion"])

        


./faces


In [7]:
import cv2
import numpy as np
import FaceNormalizationUtils as faceutils
import FaceDetectors
import random
import time
import pygame
from deepface import DeepFace


# Global variables
changing_images = False
last_image_change_time = time.time()
image_change_duration = 5  # Change the image every 10 seconds
current_image_index = 0
time_remaining = image_change_duration  # Initialize time_remaining
showImageFlag = False
onclick_frame_captured = None
elemento = None
cont = 0
race_text = None
# Function to handle mouse click events
def on_mouse_click(event, x, y, flags, param):
    
    global changing_images, last_image_change_time, time_remaining, showImageFlag, onclick_frame_captured,cont
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        showImageFlag = True
        changing_images = True
        last_image_change_time = time.time()
        time_remaining = image_change_duration  # Reset time_remaining
        onclick_frame_captured = frame
        cont = 1


# Normalizador HS
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Inicializa la captura de la webcam
cap = cv2.VideoCapture(0)

# Variables para el manejo de la ruleta de imágenes
image_paths = ['africa.png', 'america.png', 'asian.jpg','europa.jpg',"latino.png"]  
songs_paths =[
    "x2mate.com - The Lion King Opening Song Lyrics (128 kbps).mp3",
    'x2mate.com - Creedence Clearwater Revival - Fortunate Son (Sub. Español _ Lyrics) (128 kbps).mp3',
    'Xue huā piāo piāo yi jian mei - Meme (Llama, dog, frog dancing) [4K]_ue07acmWGNY.mp3',
    "x2mate.com - British Grenadiers ~ Lyrics ~ Sub. Español ~ Letra (128 kbps).mp3",
    "Suavemente Elvis Crespo Letra_B3pPGy-G_Gg.mp3",

]

# Initialize roulette_image before entering the while loop
roulette_image = cv2.imread(image_paths[current_image_index])

pygame.mixer.init()

# Create a window to capture mouse events
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', on_mouse_click)

while True:
    ret, frame = cap.read()

    if not ret:
        break
    try:
        # Calculate time remaining until image change
        time_remaining = max(0, image_change_duration - (time.time() - last_image_change_time))

        # Detección de rostros en el frame de la webcam
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
        # deepface 
        if onclick_frame_captured is not None and len(onclick_frame_captured.shape) > 1 and cont == 1 :
            obj = DeepFace.analyze(frame, enforce_detection=False, actions =['race'])
            elemento = obj[0]
            # Convertir el porcentaje a formato legible (dos decimales)
            race_text = "you are " + elemento["dominant_race"] + " by: " + "{:.2f}".format(int(elemento["race"][elemento["dominant_race"]]))+"%"

            cont = 0

        if values is not None:
            face = values[0]
            x, y, w, h = face
            if w > 0 and h > 0:
                if roulette_image is not None:
                    # Calculate the position for the superimposed face image above the detected face
                    y_offset = max(0, y - h)  # Place it above the detected face
                    x_offset = x

                    # Resize the face image to match the size of the detected face
                    resized_face_image = cv2.resize(roulette_image, (w, h))

                    
                    # Texto fuera del if
                    text_width, text_height = cv2.getTextSize( f"Which continent do you belong to?", cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                

                    # Dibujar un rectángulo blanco
                    cv2.rectangle(frame, (x_offset - 125 - 90, y_offset - text_height - 40),
                                (x_offset + text_width  - 190,  y_offset - 10), (255, 255, 255), -1)
                    cv2.rectangle(frame, (x_offset - 125 - 90, y_offset - text_height - 40),
                                (x_offset + text_width  - 190,  y_offset - 10), (0, 0, 0), 2)



                    cv2.putText(frame, f"Which continent do you belong to?", (x_offset - 125 - 75, y_offset - 25), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

                    # Mostrar race_text debajo de la imagen con suficiente separación
                    if race_text is not None and changing_images == False:
                        # Ajustar la posición del texto debajo de la imagen
                        text_size = cv2.getTextSize(race_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                        #rectangulo contenedor
                        text_width, text_height = cv2.getTextSize( race_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                        # Dibujar un rectángulo blanco
                        cv2.rectangle(frame, (x_offset - 125 - 90, y_offset + h + text_size[1] + 200),
                                    (x_offset + text_width  - 190,  y_offset + h + text_size[1] + 140), (255, 255, 255), -1)
                        #contorno rectangulo
                        cv2.rectangle(frame, (x_offset - 125 - 90, y_offset + h + text_size[1] + 200),
                                    (x_offset + text_width  - 190,  y_offset + h + text_size[1] + 140), (0, 0, 0), 2)
                        
                        
                        # Dibujar el texto
                        cv2.putText(frame, race_text, (x_offset - 125 - 75,  y_offset + h + text_size[1] + 180 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)


                    # Superpose the resized face image on the frame
                    if showImageFlag:
                        frame[y_offset:y_offset + h, x_offset:x_offset + w] = resized_face_image

        
        if changing_images:

            if current_image_index == len(image_paths):
                current_image_index = 0
            if time_remaining == 0:
                changing_images = False
                last_image_change_time = time.time()
                if(elemento["dominant_race"] == "asian"):
                    current_image_index = 2
                elif(elemento["dominant_race"] == "middle eastern"):
                    current_image_index = 1
                elif(elemento["dominant_race"] == "latino hispanic"):
                    current_image_index = 4
                elif(elemento["dominant_race"] == "white"):
                    current_image_index = 3
                elif(elemento["dominant_race"] == "black"):
                    current_image_index = 0
                elif(elemento["dominant_race"] == "indian"):
                    current_image_index = 2                
                pygame.mixer.music.load(songs_paths[current_image_index])  # Replace with your music file
                pygame.mixer.music.play(-1)
            roulette_image = cv2.imread(image_paths[current_image_index])
            current_image_index += 1
    except ValueError as e:
        print("bromita")
        pass

    cv2.imshow('Webcam', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Press Esc to exit
        # Stop the music when your program is done
        pygame.mixer.music.stop()
        break

cap.release()
cv2.destroyAllWindows()


Action: race: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


bromita
bromita
bromita


Action: race: 100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


bromita
bromita
bromita
bromita
bromita


Action: race: 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
bromita
